## 1. Import required packages

In [ ]:
from Models import run_F4
from Data.Data_Parser import fetch_all_dataset
from Solution.Map_Printer import osm_print_cluster_tours, COLOR_LIST
from Cluster_MILP import preprocess_clusters, calculate_arcs, fetch_arcs, cluster_MILP
from cluster import kmeans, kmedoids, kmeans_DBSCAN, som_cluster, p_median, recenter_clusters, convert_cluster_centers
from IPython.display import IFrame
import pickle
import json
import math
import time
import numpy as np
import pandas as pd
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

## Review input data

In [ ]:
data, vertices, arcs, V_d, V_c, F, alpha, K, Q, demands = fetch_all_dataset("LA")

In [ ]:
cluster_result = p_median(data, V_c, 9, runtime_limit=1800, print_out=False)
# cluster_result = kmeans(data) # new center
# cluster_result = kmedoids(data, 10)
# cluster_result = kmeans_DBSCAN(data) # new center

# df = pd.read_csv(".\\Data\\Shanghai.nodes", sep='\t')
# cluster_result = som_cluster(df, map_shape=(3, 3), num_iteration=100000)

## 2. Traditional home delivery model

In [ ]:
current_index = len(result_df)
# input: 'Paris' for Paris dataset, 'Shanghai' for Shanghai dataset, and 'LA' for USA dataset
data, vertices, arcs, V_d, V_c, F, alpha, K, Q, demands = fetch_all_dataset("Paris")
start_time = time.time()

objective_value, solution, runtime, mip_gap = run_F4(
    demands, vertices, arcs, V_d, V_c, F, alpha, K, Q,
    runtime_limit=1800, gap_acceptance=0.01, print_out=True
)

result_dict = {
    "obj_value": objective_value,
    "runtime": runtime,
    "mip_gap": mip_gap,
    "solution": solution
}
# input: 'Paris' for Paris dataset, 'Shanghai' for Shanghai dataset, and 'LA' for USA dataset
result_df.loc[current_index, 'dataset'] = 'Paris'
result_df.loc[current_index, 'cluster_method'] = "None"
result_df.loc[current_index, 'R'] = "-"
result_df.loc[current_index, 'num_clusters'] = "-"
result_df.loc[current_index, 'num_deli_nodes'] = len(V_c)
result_df.loc[current_index, 'milp_obj'] = result_dict['obj_value']
result_df.loc[current_index, 'milp_mip_gap'] = result_dict['mip_gap']
result_df.loc[current_index, 'milp_runtime'] = result_dict['runtime']

result_df.loc[current_index, 'avg_cluster_size'] = "-"
result_df.loc[current_index, 'avg_cluster_cost'] = "-"
result_df.loc[current_index, 'total_cost'] = result_dict['obj_value']
result_df.loc[current_index, 'total_runtime'] = time.time() - start_time
# input: 'Paris' for Paris dataset, 'Shanghai' for Shanghai dataset, and 'LA' for USA dataset
result_df.to_csv("Paris_Result.csv", index=False)

## 3. Model with combination of packstations and customers

### Set  R and K

In [ ]:
# R_list = np.linspace(2.0, 5.0, num=7)
R_list = np.array(range(2, 6))
K_clusters = np.array(range(5, 11))

### Reset dataframe for the output

In [ ]:
result_df = pd.DataFrame()

### Clustering method: Kmeans

In [ ]:
for i in tqdm(R_list):
    for j in tqdm(K_clusters):
        current_index = len(result_df)
        
        data, vertices, arcs, V_d, V_c, F, alpha, K, Q, demands = fetch_all_dataset("Paris")
        start_time = time.time()
        
        cluster_result = kmeans(data, j)
        
        result_dict, _vertices, _demands = cluster_MILP(
            data, vertices, arcs, F, alpha, K, Q, demands, 
            cluster_result, i, runtime_limit=1800, gap_acceptance=0.01, print_out=False, fast_cal=False
        )        

        result_df.loc[current_index, 'dataset'] = 'Paris'
        result_df.loc[current_index, 'cluster_method'] = "Kmeans"
        result_df.loc[current_index, 'R'] = i
        result_df.loc[current_index, 'num_clusters'] = j
        result_df.loc[current_index, 'num_deli_nodes'] = len(_vertices) - 1
        result_df.loc[current_index, 'milp_obj'] = result_dict['obj_value']
        result_df.loc[current_index, 'milp_mip_gap'] = result_dict['mip_gap']
        result_df.loc[current_index, 'milp_runtime'] = result_dict['runtime']

        total_size = 0
        total_cost = 0
        for v in [v for v in _vertices if v.startswith('P')]:
            total_cost += 8.3
            if _demands.get(v) > 4000:
                total_cost += (_demands.get(v) - 4000) * 0.002
            total_size += _demands.get(v)
        
        result_df.loc[current_index, 'avg_cluster_size'] = total_size / j
        result_df.loc[current_index, 'avg_cluster_cost'] = total_cost / j
        result_df.loc[current_index, 'total_cost'] = total_cost + result_dict['obj_value']
        result_df.loc[current_index, 'total_runtime'] = time.time() - start_time
        result_df.to_csv("Paris_Result.csv", index=False)

### Clustering method: DBSCAN

In [ ]:
for i in tqdm(R_list):
#     if i == R_list[0]:
#         continue
    current_index = len(result_df)

    data, vertices, arcs, V_d, V_c, F, alpha, K, Q, demands = fetch_all_dataset("Paris")
    start_time = time.time()

    cluster_result = kmeans_DBSCAN(data)

    result_dict, _vertices, _demands = cluster_MILP(
        data, vertices, arcs, F, alpha, K, Q, demands, 
        cluster_result, i, runtime_limit=1800, gap_acceptance=0.01, print_out=False, fast_cal=False
    )        

    result_df.loc[current_index, 'dataset'] = 'Paris'
    result_df.loc[current_index, 'cluster_method'] = "kmeans_DBSCAN"
    result_df.loc[current_index, 'R'] = i
    result_df.loc[current_index, 'num_clusters'] = len(cluster_result.items())
    result_df.loc[current_index, 'num_deli_nodes'] = len(_vertices) - 1
    result_df.loc[current_index, 'milp_obj'] = result_dict['obj_value']
    result_df.loc[current_index, 'milp_mip_gap'] = result_dict['mip_gap']
    result_df.loc[current_index, 'milp_runtime'] = result_dict['runtime']

    total_size = 0
    total_cost = 0
    for v in [v for v in _vertices if v.startswith('P')]:
        total_cost += 8.3
        if _demands.get(v) > 4000:
            total_cost += (_demands.get(v) - 4000) * 0.002
        total_size += _demands.get(v)

    result_df.loc[current_index, 'avg_cluster_size'] = total_size / len(cluster_result.items())
    result_df.loc[current_index, 'avg_cluster_cost'] = total_cost / len(cluster_result.items())
    result_df.loc[current_index, 'total_cost'] = total_cost + result_dict['obj_value']
    result_df.loc[current_index, 'total_runtime'] = time.time() - start_time
    result_df.to_csv("Paris_Result.csv", index=False)

### Clustering method: P median

In [ ]:
for i in tqdm(R_list):
    for j in tqdm(K_clusters):
        current_index = len(result_df)
        
        data, vertices, arcs, V_d, V_c, F, alpha, K, Q, demands = fetch_all_dataset("Paris")
        start_time = time.time()
        
        cluster_result = p_median(data, V_c, j, runtime_limit=1800, print_out=False)
        
        result_dict, _vertices, _demands = cluster_MILP(
            data, vertices, arcs, F, alpha, K, Q, demands, 
            cluster_result, i, runtime_limit=1800, gap_acceptance=0.01, print_out=False, fast_cal=False
        )        

        result_df.loc[current_index, 'dataset'] = 'Paris'
        result_df.loc[current_index, 'cluster_method'] = "p_median"
        result_df.loc[current_index, 'R'] = i
        result_df.loc[current_index, 'num_clusters'] = j
        result_df.loc[current_index, 'num_deli_nodes'] = len(_vertices) - 1
        result_df.loc[current_index, 'milp_obj'] = result_dict['obj_value']
        result_df.loc[current_index, 'milp_mip_gap'] = result_dict['mip_gap']
        result_df.loc[current_index, 'milp_runtime'] = result_dict['runtime']

        total_size = 0
        total_cost = 0
        for v in [v for v in _vertices if v.startswith('P')]:
            total_cost += 8.3
            if _demands.get(v) > 4000:
                total_cost += (_demands.get(v) - 4000) * 0.002
            total_size += _demands.get(v)
        
        result_df.loc[current_index, 'avg_cluster_size'] = total_size / j
        result_df.loc[current_index, 'avg_cluster_cost'] = total_cost / j
        result_df.loc[current_index, 'total_cost'] = total_cost + result_dict['obj_value']
        result_df.loc[current_index, 'total_runtime'] = time.time() - start_time
        result_df.to_csv("Paris_Result_pmedian.csv", index=False)

### Clustering method: Kmedoids

In [ ]:
for i in tqdm(R_list):
    for j in tqdm(K_clusters):
        current_index = len(result_df)
        
        data, vertices, arcs, V_d, V_c, F, alpha, K, Q, demands = fetch_all_dataset("Paris")
        start_time = time.time()
        
        cluster_result = kmedoids(data, j)
        
        result_dict, _vertices, _demands = cluster_MILP(
            data, vertices, arcs, F, alpha, K, Q, demands, 
            cluster_result, i, runtime_limit=1800, gap_acceptance=0.01, print_out=False, fast_cal=False
        )        

        result_df.loc[current_index, 'dataset'] = 'Paris'
        result_df.loc[current_index, 'cluster_method'] = "kmedoids"
        result_df.loc[current_index, 'R'] = i
        result_df.loc[current_index, 'num_clusters'] = j
        result_df.loc[current_index, 'num_deli_nodes'] = len(_vertices) - 1
        result_df.loc[current_index, 'milp_obj'] = result_dict['obj_value']
        result_df.loc[current_index, 'milp_mip_gap'] = result_dict['mip_gap']
        result_df.loc[current_index, 'milp_runtime'] = result_dict['runtime']

        total_size = 0
        total_cost = 0
        for v in [v for v in _vertices if v.startswith('P')]:
            total_cost += 8.3
            if _demands.get(v) > 4000:
                total_cost += (_demands.get(v) - 4000) * 0.002
            total_size += _demands.get(v)
        
        result_df.loc[current_index, 'avg_cluster_size'] = total_size / j
        result_df.loc[current_index, 'avg_cluster_cost'] = total_cost / j
        result_df.loc[current_index, 'total_cost'] = total_cost + result_dict['obj_value']
        result_df.loc[current_index, 'total_runtime'] = time.time() - start_time
        result_df.to_csv("Paris_Result.csv", index=False)

### Clustering method: SOM

In [ ]:
K_shape = [(1, 5), (2, 3), (1, 7), (2, 4), (3, 3), (2, 5)]

for i in tqdm(R_list):
    for j in tqdm(K_shape):
        current_index = len(result_df)
        if i == R_list[0] and j == K_shape[0]:
            continue
        
        data, vertices, arcs, V_d, V_c, F, alpha, K, Q, demands = fetch_all_dataset("Paris")
        start_time = time.time()
        
        df = pd.read_csv(".\\Data\\Paris.nodes", sep=' ')
        cluster_result = som_cluster(df, map_shape=j, num_iteration=100000)
        
        result_dict, _vertices, _demands = cluster_MILP(
            data, vertices, arcs, F, alpha, K, Q, demands, 
            cluster_result, i, runtime_limit=1800, gap_acceptance=0.01, print_out=False, fast_cal=False
        )        

        result_df.loc[current_index, 'dataset'] = 'Paris'
        result_df.loc[current_index, 'cluster_method'] = "SOM"
        result_df.loc[current_index, 'R'] = i
        result_df.loc[current_index, 'num_clusters'] = j[0] * j[1]
        result_df.loc[current_index, 'num_deli_nodes'] = len(_vertices) - 1
        result_df.loc[current_index, 'milp_obj'] = result_dict['obj_value']
        result_df.loc[current_index, 'milp_mip_gap'] = result_dict['mip_gap']
        result_df.loc[current_index, 'milp_runtime'] = result_dict['runtime']

        total_size = 0
        total_cost = 0
        for v in [v for v in _vertices if v.startswith('P')]:
            total_cost += 8.3
            if _demands.get(v) > 4000:
                total_cost += (_demands.get(v) - 4000) * 0.002
            total_size += _demands.get(v)
        
        result_df.loc[current_index, 'avg_cluster_size'] = total_size / (j[0] * j[1])
        result_df.loc[current_index, 'avg_cluster_cost'] = total_cost / (j[0] * j[1])
        result_df.loc[current_index, 'total_cost'] = total_cost + result_dict['obj_value']
        result_df.loc[current_index, 'total_runtime'] = time.time() - start_time
        result_df.to_csv("Paris_Result.csv", index=False)

In [ ]:
result_df